In [ ]:
import copy
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep

from boostedhh import utils, hh_vars, plotting
from boostedhh.utils import PAD_VAL
from bbtautau import bbtautau_vars

import SensitivityStudy

import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("boostedhh.utils")
logger.setLevel(logging.DEBUG)

plt.style.use(hep.style.CMS)
hep.style.use("CMS")

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_DIR = Path("../../../")
CHANNEL = "electron"  # options: "hadronic", "electron", "muon"

plot_dir = MAIN_DIR / f"plots/SensitivityStudy/25Mar7{CHANNEL}"
plot_dir.mkdir(parents=True, exist_ok=True)

years = ["2022"]  # ["2023","2023BPix"]  # "2022","2022EE",

tags = {
    "data": {
        "2022": "24Nov21ParTMass_v12_private_signal",
        "2022EE": "25Jan22AddYears_v12_private_signal",
        "2023": "25Mar7Signal_v12_private_signal",
        "2023BPix": "25Mar7Signal_v12_private_signal",
    },
    "signal": {
        "2022": "24Nov21ParTMass_v12_private_signal",
        "2022EE": "25Jan22AddYears_v12_private_signal",
        "2023": "25Mar7Signal_v12_private_signal",
        "2023BPix": "25Mar7Signal_v12_private_signal",
    },
}

base_dir = {
    "2022": Path("/ceph/cms/store/user/rkansal/bbtautau/skimmer/"),
    "2022EE": Path("/ceph/cms/store/user/rkansal/bbtautau/skimmer/"),
    "2023": Path("/ceph/cms/store/user/lumori/bbtautau/skimmer/"),
    "2023BPix": Path("/ceph/cms/store/user/lumori/bbtautau/skimmer/"),
}

In [ ]:
# pick signal key based on channel
SIG_KEYS = {"hadronic": "bbtthh", "electron": "bbtthe", "muon": "bbtthmu"}
SIG_KEY = SIG_KEYS[CHANNEL]

# pick relevant data samples based on channel
DATA_KEYS = {
    "hadronic": ["jetmet", "tau"],
    "electron": ["jetmet", "tau", "egamma"],
    "muon": ["jetmet", "tau", "muon"],
}[CHANNEL]

# pick relevant lepton dataset based on channel
LEPTON_DATASET = {"hadronic": None, "electron": "egamma", "muon": "muon"}[CHANNEL]

ALL_TRIGGERS = {
    "hadronic": bbtautau_vars.HLT_hh,
    "muon": bbtautau_vars.HLT_hmu,
    "electron": bbtautau_vars.HLT_he,
}

LEPTON_TRIGGERS = {
    "hadronic": None,
    "electron": bbtautau_vars.HLT_he,
    "muon": bbtautau_vars.HLT_hmu,
}

In [ ]:
analyser = SensitivityStudy.Analyser(years, CHANNEL)
for year in years:
    analyser.extract_year(year)
    print(f"Loaded {year} events")

In [ ]:
qcdouts = ["QCD0HF", "QCD1HF", "QCD2HF"]
topouts = ["TopW", "TopbW", "TopbWev", "TopbWmv", "TopbWtauhv", "TopbWq", "TopbWqq"]
sigouts = ["Xtauhtauh", "Xtauhtaue", "Xtauhtaum", "Xbb"]

columns_data = {
    year: [
        ("weight", 1),
        ("ak8FatJetMsd", 3),
        ("ak8FatJetEta", 3),
        ("ak8FatJetPt", 3),
        ("ak8FatJetPhi", 3),
        ("ak8FatJetPNetXbbLegacy", 3),
        ("ak8FatJetPNetQCDLegacy", 3),
        ("ak8FatJetPNetmassLegacy", 3),
        ("ak8FatJetParTmassResApplied", 3),
        ("ak8FatJetParTmassVisApplied", 3),
    ]
    for year in years
}


columns_signal = copy.deepcopy(columns_data)

for year in years:
    for branch in ALL_TRIGGERS[CHANNEL][year]:
        columns_data[year].append((branch, 1))
        columns_signal[year].append((branch, 1))

    for branch in [f"ak8FatJetParT{key}" for key in qcdouts + topouts + sigouts]:
        columns_data[year].append((branch, 3))
        columns_signal[year].append((branch, 3))

    columns_signal[year] += [
        ("GenTauhh", 1),
        ("GenTauhmu", 1),
        ("GenTauhe", 1),
        ("GenHiggsEta", 2),
        ("GenHiggsPt", 2),
        ("GenHiggsPhi", 2),
        ("GenHiggsMass", 2),
        ("GenbbEta", 2),
        ("GenbbPt", 2),
        ("GenbbPhi", 2),
        ("GenbbMass", 2),
        ("GenTauEta", 2),
        ("GenTauPt", 2),
        ("GenTauPhi", 2),
        ("GenTauMass", 2),
        ("single_weight_genweight", 1),
        ("single_weight_pileup", 1),
        ("single_weight_ISRPartonShower", 1),
        ("single_weight_FSRPartonShower", 1),
    ]

## Define and load samples

In [ ]:
# define samples to load
samples = {
    year: {
        "jetmet": utils.Sample(
            path=base_dir[year] / tags["data"][year],
            selector="JetHT|JetMET",
            label="JetMET",
            isData=True,
            year=year,
            load_columns=utils.format_columns(columns_data[year]),
        ),
        "tau": utils.Sample(
            path=base_dir[year] / tags["data"][year],
            selector="Tau_Run",
            label="Tau",
            isData=True,
            year=year,
            load_columns=utils.format_columns(columns_data[year]),
        ),
        "muon": utils.Sample(
            path=base_dir[year] / tags["data"][year],
            selector="Muon_Run",
            label="Muon",
            isData=True,
            year=year,
            load_columns=utils.format_columns(columns_data[year]),
        ),
        "egamma": utils.Sample(
            path=base_dir[year] / tags["data"][year],
            selector="EGamma_Run",
            label="EGamma",
            isData=True,
            year=year,
            load_columns=utils.format_columns(columns_data[year]),
        ),
        "bbtt": utils.Sample(
            path=base_dir[year] / tags["signal"][year],
            selector=hh_vars.bbtt_sigs["bbtt"][year],
            label=r"HHbb$\tau\tau$",
            isData=False,
            year=year,
            load_columns=utils.format_columns(columns_signal[year]),
        ),
    }
    for year in years
}

for key in ["jetmet", "tau", "egamma", "muon"]:
    if key not in DATA_KEYS:
        for year in years:
            del samples[year][key]

In [ ]:
# pt_cut = 250
# msd_cut = 40

filters = [
    [
        ("('ak8FatJetPt', '0')", ">=", 250),
        ("('ak8FatJetPNetmassLegacy', '0')", ">=", 50),
        ("('ak8FatJetPt', '1')", ">=", 200),
        # ("('ak8FatJetMsd', '0')", ">=", msd_cut),
        # ("('ak8FatJetMsd', '1')", ">=", msd_cut),
        # ("('ak8FatJetPNetXbb', '0')", ">=", 0.8),
    ],
]


# dictionary that will contain all information (from all samples)
events_dict = {year: {} for year in years}

for year in years:
    for key, sample in samples[year].items():
        events_dict[year][key] = utils.load_sample(sample, filters)

    events_dict[year]["bbtthh"] = events_dict[year]["bbtt"][
        events_dict[year]["bbtt"]["GenTauhh"][0]
    ]
    events_dict[year]["bbtthmu"] = events_dict[year]["bbtt"][
        events_dict[year]["bbtt"]["GenTauhmu"][0]
    ]
    events_dict[year]["bbtthe"] = events_dict[year]["bbtt"][
        events_dict[year]["bbtt"]["GenTauhe"][0]
    ]
    del events_dict[year]["bbtt"]


cutflow = {year: pd.DataFrame(index=list(events_dict[year].keys())) for year in years}

for year in years:
    utils.add_to_cutflow(events_dict[year], "Preselection", "finalWeight", cutflow[year])

## Triggers

In [ ]:
for year in years:
    print(year)
    for skey in SIG_KEYS.values():
        print(skey)
        triggered = np.sum(
            [events_dict[year][skey][hlt].iloc[:, 0] for hlt in ALL_TRIGGERS[CHANNEL][year]],
            axis=0,
        ).astype(bool)
        events_dict[year][skey] = events_dict[year][skey][triggered]

### Data (overlap removal)

In [ ]:
trigdict = {year: {"jetmet": {}, "tau": {}} for year in years}

if LEPTON_DATASET:
    for year in years:
        trigdict[year][LEPTON_DATASET] = {}

for year in years:
    for key, d in trigdict[year].items():
        d["all"] = np.sum(
            [events_dict[year][key][hlt].iloc[:, 0] for hlt in ALL_TRIGGERS[CHANNEL][year]], axis=0
        ).astype(bool)
        d["jets"] = np.sum(
            [
                events_dict[year][key][hlt].iloc[:, 0]
                for hlt in bbtautau_vars.HLT_dict[year]["PNet"]
                + bbtautau_vars.HLT_dict[year]["PFJet"]
            ],
            axis=0,
        ).astype(bool)
        d["taus"] = np.sum(
            [events_dict[year][key][hlt].iloc[:, 0] for hlt in bbtautau_vars.HLT_taus[year]], axis=0
        ).astype(bool)

        d["taunojets"] = ~d["jets"] & d["taus"]

        if LEPTON_DATASET:
            d[LEPTON_DATASET] = np.sum(
                [events_dict[year][key][hlt].iloc[:, 0] for hlt in LEPTON_TRIGGERS[CHANNEL][year]],
                axis=0,
            ).astype(bool)

            d[f"{LEPTON_DATASET}noothers"] = ~d["jets"] & ~d["taus"] & d[LEPTON_DATASET]

Checking event loss by flipping triggers (can skip)

In [ ]:
# xor = np.setdiff1d(
#     events_dict["jetmet"][trigdict["jetmet"]["nojettau"]]["event"][0],
#     events_dict["tau"][trigdict["tau"]["nojettau"]]["event"][0],
# )

# print(len(xor) / len(events_dict["jetmet"]))

# xor = np.setdiff1d(
#     events_dict["tau"][trigdict["tau"]["jetnotau"]]["event"][0],
#     events_dict["jetmet"][trigdict["jetmet"]["jets"]]["event"][0],
# )

# print(len(xor) / len(events_dict["tau"]))

Apply overlap removal

In [ ]:
for year in years:
    events_dict[year]["jetmet"] = events_dict[year]["jetmet"][trigdict[year]["jetmet"]["jets"]]
    events_dict[year]["tau"] = events_dict[year]["tau"][trigdict[year]["tau"]["taunojets"]]
    if LEPTON_DATASET:
        events_dict[year][LEPTON_DATASET] = events_dict[year][LEPTON_DATASET][
            trigdict[year][LEPTON_DATASET][f"{LEPTON_DATASET}noothers"]
        ]

In [ ]:
for year in years:
    utils.add_to_cutflow(events_dict[year], "Triggers", "finalWeight", cutflow[year])
cutflow

## FatJet Gen Matching

In [ ]:
# merge years in gen matching
events = pd.concat([events_dict[year][SIG_KEY] for year in years], keys=[year for year in years])

higgs = utils.make_vector(events, "GenHiggs")
bb = utils.make_vector(events, "Genbb")
tt = utils.make_vector(events, "GenTau")
fatjets = utils.make_vector(events, "ak8FatJet", mstring="Msd")

events

In [ ]:
minbb = np.min(higgs[:, 0:1].deltaR(bb), axis=1)
mintau = np.min(higgs[:, 0:1].deltaR(tt), axis=1)
genhbb1 = minbb < mintau

# minbb = np.min(higgs[:, 1:2].deltaR(bb), axis=1)
# mintau = np.min(higgs[:, 1:2].deltaR(tt), axis=1)
# genhbb2 = minbb < mintau  # overlap with genhb1 < 0.5% of the time

genhbb_mask = np.vstack([genhbb1, ~genhbb1]).T
genhbb = higgs[genhbb_mask]
genhtt = higgs[~genhbb_mask]

In [ ]:
fjbbdr = fatjets.deltaR(genhbb[:, np.newaxis])
fjidbb = np.argmin(fjbbdr, axis=1)
fjttdr = fatjets.deltaR(genhtt[:, np.newaxis])
fjidtt = np.argmin(fjttdr, axis=1)
# 5% of events have overlap out of which only 5% actually have two jets both close to a gen Higgs,
# so ignoring these overlap events for now
overlap = fjidbb == fjidtt

In [ ]:
np.mean(overlap)

## Taggers

In [ ]:
taggers_dict = {year: {} for year in years}

taukey = {"hadronic": "Xtauhtauh", "electron": "Xtauhtaue", "muon": "Xtauhtaum"}[CHANNEL]

for year in years:
    for key, events in events_dict[year].items():
        tvars = {}

        qcdouts = ["QCD0HF", "QCD1HF", "QCD2HF"]  # HF = heavy flavor = {c,b}
        topouts = ["TopW", "TopbW"]  # "TopbWev", "TopbWmv", "TopbWtauhv", "TopbWq", "TopbWqq"]
        tvars["PQCD"] = sum([events[f"ak8FatJetParT{k}"] for k in qcdouts]).to_numpy()
        tvars["PTop"] = sum([events[f"ak8FatJetParT{k}"] for k in topouts]).to_numpy()

        for disc in ["Xbb", taukey]:
            tvars[f"{disc}vsQCD"] = np.nan_to_num(
                events[f"ak8FatJetParT{disc}"] / (events[f"ak8FatJetParT{disc}"] + tvars["PQCD"]),
                nan=PAD_VAL,
            )
            tvars[f"{disc}vsQCDTop"] = np.nan_to_num(
                events[f"ak8FatJetParT{disc}"]
                / (events[f"ak8FatJetParT{disc}"] + tvars["PQCD"] + tvars["PTop"]),
                nan=PAD_VAL,
            )

            # make sure not to choose padded jets below by accident
            nojet3 = events["ak8FatJetPt"][2] == PAD_VAL
            tvars[f"{disc}vsQCD"][:, 2][nojet3] = PAD_VAL
            tvars[f"{disc}vsQCDTop"][:, 2][nojet3] = PAD_VAL

        tvars["PNetXbbvsQCD"] = np.nan_to_num(
            events["ak8FatJetPNetXbbLegacy"]
            / (events["ak8FatJetPNetXbbLegacy"] + events["ak8FatJetPNetQCDLegacy"]),
            nan=PAD_VAL,
        )

        # jet assignment
        fjbbpick = np.argmax(tvars["XbbvsQCD"], axis=1)
        fjttpick = np.argmax(tvars[f"{taukey}vsQCD"], axis=1)
        overlap = fjbbpick == fjttpick
        fjbbpick[overlap] = np.argsort(tvars["XbbvsQCD"][overlap], axis=1)[:, -2]

        # convert ids to boolean masks
        fjbbpick_mask = np.zeros_like(tvars["XbbvsQCD"], dtype=bool)
        fjbbpick_mask[np.arange(len(fjbbpick)), fjbbpick] = True
        fjttpick_mask = np.zeros_like(tvars[f"{taukey}vsQCD"], dtype=bool)
        fjttpick_mask[np.arange(len(fjttpick)), fjttpick] = True

        tvars["bb_mask"] = fjbbpick_mask
        tvars["tautau_mask"] = fjttpick_mask
        taggers_dict[year][key] = tvars

Checking bb matching accuracy (can skip)

In [ ]:
tvars = {}
if len(years) > 1:
    for key in taggers_dict[years[0]][SIG_KEY].keys():
        tvars[key] = np.concatenate([taggers_dict[year][SIG_KEY][key] for year in years])
else:
    tvars = taggers_dict[years[0]][SIG_KEY]

maxtxbb = np.max(tvars["XbbvsQCD"], axis=1)
fjbbpick = np.argmax(tvars["XbbvsQCD"], axis=1)
maxtxtt = np.max(tvars[f"{taukey}vsQCD"], axis=1)
fjttpick = np.argmax(tvars[f"{taukey}vsQCD"], axis=1)

# how many are assigned correctly?
print(f"Correct matching bb: {np.mean(fjbbpick == fjidbb)}")
print(f"Correct matching tt: {np.mean(fjttpick == fjidtt)}")

overlap = fjbbpick == fjttpick
print(f"Overlap: {np.mean(overlap)}")
# how many pass reasonable tagger cuts?
print(
    f"How many overlaps pass basic tagger cuts: {np.sum((maxtxbb > 0.8) * (maxtxtt > 0.95) * overlap) / np.sum(overlap)}"
)  # <0.1%

In [ ]:
def get_jet_vals(vals, mask):
    # check if vals is a numpy array
    if not isinstance(vals, np.ndarray):
        vals = vals.to_numpy()

    return vals[mask]

### ROC Curves

In [ ]:
from sklearn.metrics import roc_curve


def compute_rocs(
    years,
    jets=["bb", "tautau"],
    discs=["XbbvsQCD", "XbbvsQCDTop", f"{taukey}vsQCD", f"{taukey}vsQCDTop", "PNetXbbvsQCD"],
):

    rocs = {}

    for jet in jets:
        print(jet)
        rocs[jet] = {}
        for i, disc in enumerate(discs):
            print("\t" + disc)

            bg_scores = np.concatenate(
                [
                    get_jet_vals(
                        taggers_dict[year][key][disc], taggers_dict[year][key][f"{jet}_mask"]
                    )
                    for key in DATA_KEYS
                    for year in years
                ]
            )
            bg_weights = np.concatenate(
                [events_dict[year][key]["finalWeight"] for key in DATA_KEYS for year in years]
            )

            sig_scores = np.concatenate(
                [
                    get_jet_vals(
                        taggers_dict[year][SIG_KEY][disc],
                        taggers_dict[year][SIG_KEY][f"{jet}_mask"],
                    )
                    for year in years
                ]
            )
            sig_weights = np.concatenate(
                [events_dict[year][SIG_KEY]["finalWeight"] for year in years]
            )

            fpr, tpr, thresholds = roc_curve(
                np.concatenate([np.zeros_like(bg_scores), np.ones_like(sig_scores)]),
                np.concatenate([bg_scores, sig_scores]),
                sample_weight=np.concatenate([bg_weights, sig_weights]),
            )

            rocs[jet][disc] = {
                "fpr": fpr,
                "tpr": tpr,
                "thresholds": thresholds,
                "label": disc,
                "color": plt.cm.tab10.colors[i],
            }

    return rocs

In [ ]:
rocs_full = compute_rocs(years)

In [ ]:
for jet, title in zip(["bb", "tautau"], ["bb FatJet", rf"$\tau_h\tau_{CHANNEL[0]}$ FatJet"]):
    if len(years) > 1:
        for i, year in enumerate(years):
            if i == 0:
                title += f" {year}"
            else:
                title += f"+{year}"
    else:
        title = title + f" {years[0]}"

    plotting.multiROCCurveGrey(
        {"": rocs_full[jet]},
        title=title,
        show=True,
        plot_dir=plot_dir,
        name=f"roc_{jet} {years}",
    )

In [ ]:
rocs = {year: compute_rocs([year]) for year in years}

In [ ]:
for year in years:
    for jet, title in zip(["bb", "tautau"], ["bb FatJet", rf"$\tau_h\tau_{CHANNEL[0]}$ FatJet"]):
        plotting.multiROCCurveGrey(
            {"": rocs[year][jet]},
            title=title + " " + year,
            show=True,
            plot_dir=plot_dir,
            name=f"roc_{jet} {year}",
        )

## Mass

In [ ]:
for year in years:
    for key, label in zip(["hhbbtt", "data"], ["HHbbtt", "Data"]):
        if key == "hhbbtt":
            events = events_dict[year][SIG_KEY]
        else:
            events = pd.concat([events_dict[year][dkey] for dkey in DATA_KEYS])

        bins = np.linspace(0, 250, 50)

        fig, axs = plt.subplots(1, 2, figsize=(24, 10))

        for i, (jet, jlabel) in enumerate(
            zip(["bb", "tautau"], ["bb FatJet", r"$\tau\tau$ FatJet"])
        ):
            ax = axs[i]
            if key == "hhbbtt":
                mask = taggers_dict[year][SIG_KEY][f"{jet}_mask"]
            else:
                mask = np.concatenate(
                    [taggers_dict[year][dkey][f"{jet}_mask"] for dkey in DATA_KEYS], axis=0
                )

            for j, (mkey, mlabel) in enumerate(
                zip(
                    [
                        "ak8FatJetMsd",
                        "ak8FatJetPNetmassLegacy",
                        "ak8FatJetParTmassResApplied",
                        "ak8FatJetParTmassVisApplied",
                    ],
                    ["SoftDrop", "PNetLegacy", "ParT Res", "ParT Vis"],
                )
            ):
                ax.hist(
                    get_jet_vals(events[mkey], mask),
                    bins=bins,
                    histtype="step",
                    weights=events["finalWeight"],
                    label=mlabel,
                    linewidth=2,
                    color=plt.cm.tab10.colors[j],
                )

            ax.vlines(125, 0, ax.get_ylim()[1], linestyle="--", color="k", alpha=0.1)
            ax.set_title(jlabel, fontsize=24)
            ax.set_xlabel("Mass [GeV]")
            # rax.set_xlabel("Mass [GeV]")
            ax.set_ylabel("Events")
            ax.legend()
            ax.set_ylim(0)
            hep.cms.label(
                ax=ax,
                data=key == "data",
                year=year,
                com="13.6",
                fontsize=20,
                lumi=f"{hh_vars.LUMI[year] / 1000:.1f}",
            )

        plt.savefig(plot_dir / f"{key}_{year}_mass.pdf", bbox_inches="tight")
        plt.show()

## Cut-and-count

In [ ]:
from matplotlib.lines import Line2D

# bbeff, tteff = 0.44,0.36 #0.44, 0.36 values determined by highest sig for 1 bkg event
mbb1, mbb2 = 110.0, 160.0
mbbw2 = (mbb2 - mbb1) / 2
mtt1, mtt2 = 50, 1500

# mbbk = "PNetmassLegacy"
mbbk = "ParTmassResApplied"
# mttk = "PNetmassLegacy"
mttk = "ParTmassResApplied"

txbbs = {year: {} for year in years}
txtts = {year: {} for year in years}
masstt = {year: {} for year in years}
massbb = {year: {} for year in years}
ptbb = {year: {} for year in years}

# precompute to speedup
for year in years:
    for key in [SIG_KEY] + DATA_KEYS:
        txbbs[year][key] = get_jet_vals(
            taggers_dict[year][key]["XbbvsQCD"], taggers_dict[year][key]["bb_mask"]
        )
        txtts[year][key] = get_jet_vals(
            taggers_dict[year][key][f"{taukey}vsQCDTop"], taggers_dict[year][key]["tautau_mask"]
        )
        masstt[year][key] = get_jet_vals(
            events_dict[year][key][f"ak8FatJet{mttk}"], taggers_dict[year][key]["tautau_mask"]
        )
        massbb[year][key] = get_jet_vals(
            events_dict[year][key][f"ak8FatJet{mbbk}"], taggers_dict[year][key]["bb_mask"]
        )
        ptbb[year][key] = get_jet_vals(
            events_dict[year][key]["ak8FatJetPt"], taggers_dict[year][key]["bb_mask"]
        )


def compute_sig_bg(year, txbbcut, txttcut, mbb1, mbb2, mbbw2, mtt1, mtt2):
    bg_yield = 0
    sig_yield = 0

    for key in [SIG_KEY] + DATA_KEYS:
        # plt.hist(massbb, np.linspace(0, 200, 100), histtype="step", label=key, weights=events_dict[key]["finalWeight"])
        if key == SIG_KEY:
            cut = (
                (txbbs[year][key] > txbbcut)
                & (txtts[year][key] > txttcut)
                & (masstt[year][key] > mtt1)
                & (masstt[year][key] < mtt2)
                & (massbb[year][key] > mbb1)
                & (massbb[year][key] < mbb2)
                & (ptbb[year][key] > 250)
            )
            sig_yield += np.sum(events_dict[year][key]["finalWeight"][cut])
        else:
            cut = (
                (txbbs[year][key] > txbbcut)
                & (txtts[year][key] > txttcut)
                & (masstt[year][key] > mtt1)
                & (masstt[year][key] < mtt2)
                & (ptbb[year][key] > 250)
            )
            msb1 = (massbb[year][key] > (mbb1 - mbbw2)) & (massbb[year][key] < mbb1)
            msb2 = (massbb[year][key] > mbb2) & (massbb[year][key] < (mbb2 + mbbw2))
            bg_yield += np.sum(events_dict[year][key]["finalWeight"][cut & msb1])
            bg_yield += np.sum(events_dict[year][key]["finalWeight"][cut & msb2])
    return sig_yield, bg_yield


def sig_bkg_opt(year, gridsize=100, gridlims=(0.5, 1), plot=False):
    # bbeff_vals = np.linspace(0.3, 0.7, gridsize)
    # tteff_vals = np.linspace(0.3, 0.7, gridsize)
    bbcut = np.linspace(*gridlims, gridsize)
    ttcut = np.linspace(*gridlims, gridsize)

    BBcut, TTcut = np.meshgrid(bbcut, ttcut)

    # scalar function, must be vectorized
    sig_bg = lambda bbcut, ttcut: compute_sig_bg(
        year=year,
        txbbcut=bbcut,
        txttcut=ttcut,
        # txbbcut=rocs[year]["bb"]["XbbvsQCD"]["thresholds"][
        #     plotting._find_nearest(rocs[years[0]]["bb"]["XbbvsQCD"]["tpr"], bbeff)
        # ],
        # txttcut=rocs[year]["tautau"][f"{taukey}vsQCDTop"]["thresholds"][
        #     plotting._find_nearest(rocs[years[0]]["tautau"][f"{taukey}vsQCDTop"]["tpr"], tteff)
        # ],
        mbb1=mbb1,
        mbb2=mbb2,
        mbbw2=mbbw2,
        mtt1=mtt1,
        mtt2=mtt2,
    )

    sigs, bgs = np.vectorize(sig_bg)(BBcut, TTcut)
    sel = bgs == 1

    if np.sum(sel) == 0:
        n = 2
        while np.sum(sel) == 0:
            sel = (bgs >= 1) & (bgs <= n)
            n += 1
        print(f"Need a finer grid, no region with B=1. I'm extending the region to B in [1,{n}].")

    sel_idcs = np.argwhere(sel)
    sel_sigs = sigs[sel]

    opt_i = np.argmax(sel_sigs)
    max_sig_idx = tuple(sel_idcs[opt_i])
    bbcut_opt, ttcut_opt = BBcut[max_sig_idx], TTcut[max_sig_idx]

    print(bgs)

    if plot:
        fig, ax = plt.subplots(figsize=(8, 8))
        hep.cms.label(
            ax=ax,
            data=True,
            year=year,
            com="13.6",
            fontsize=16,
            lumi=f"{hh_vars.LUMI[year] / 1000:.1f}",
        )
        sigmap = ax.contourf(BBcut, TTcut, sigs, levels=10, cmap="viridis")
        contour = ax.contour(BBcut, TTcut, sel, colors="r")
        proxy = Line2D([0], [0], color="r", label="B=1 level")
        ax.scatter(bbcut_opt, ttcut_opt, color="r", label="Optimal cut")
        ax.set_xlabel("Xbb vs QCD cut")
        ax.set_ylabel("Xtauhtauh vs QCD cut")
        cbar = plt.colorbar(sigmap, ax=ax)
        cbar.set_label("Signal yield")
        handles, labels = ax.get_legend_handles_labels()
        handles.append(proxy)
        ax.legend(handles=handles, loc="lower left")
        plt.savefig(plot_dir / f"sig_bkg_opt_{year}.pdf", bbox_inches="tight")
        plt.savefig(plot_dir / f"sig_bkg_opt_{year}.png", bbox_inches="tight")
        plt.show()

    return sigs[max_sig_idx], bgs[max_sig_idx], [bbcut_opt, ttcut_opt]


def print_nicely(sig_yield, bg_yield, years):
    print(
        f"""
        
        Yield study year(s) {years}:
        
        """
    )

    print("Sig yield", sig_yield)
    print("BG yield", bg_yield)
    print("limit", 2 * np.sqrt(bg_yield) / sig_yield)

    if "2023" not in years or "2023BPix" not in years:
        print(
            "limit scaled to 22-23 all channels",
            2
            * np.sqrt(bg_yield)
            / sig_yield
            / np.sqrt(
                hh_vars.LUMI["2022-2023"] / np.sum([hh_vars.LUMI[year] for year in years]) * 3
            ),
        )
    print(
        "limit scaled to 22-24 all channels",
        2
        * np.sqrt(bg_yield)
        / sig_yield
        / np.sqrt(
            (124000 + hh_vars.LUMI["2022-2023"])
            / np.sum([hh_vars.LUMI[year] for year in years])
            * 3
        ),
    )
    print(
        "limit scaled to Run 3 all channels",
        2 * np.sqrt(bg_yield) / sig_yield / np.sqrt((360000) / hh_vars.LUMI[year] * 3),
    )
    return


def as_df(sig_yield, bg_yield, years):
    limits = {}

    limits["Sig_Yield"] = sig_yield
    limits["BG_Yield"] = bg_yield
    limits["Limit"] = 2 * np.sqrt(bg_yield) / sig_yield

    if "2023" not in years and "2023BPix" not in years:
        limits["Limit_scaled_22_23"] = (
            2
            * np.sqrt(bg_yield)
            / sig_yield
            / np.sqrt(
                hh_vars.LUMI["2022-2023"] / np.sum([hh_vars.LUMI[year] for year in years]) * 3
            )
        )
    else:
        limits["Limit_scaled_22_23"] = np.nan

    limits["Limit_scaled_22_24"] = (
        2
        * np.sqrt(bg_yield)
        / sig_yield
        / np.sqrt(
            (124000 + hh_vars.LUMI["2022-2023"])
            / np.sum([hh_vars.LUMI[year] for year in years])
            * 3
        )
    )

    limits["Limit_scaled_Run3"] = (
        2
        * np.sqrt(bg_yield)
        / sig_yield
        / np.sqrt((360000) / np.sum([hh_vars.LUMI[year] for year in years]) * 3)
    )

    df = pd.DataFrame([limits])

    return df

In [ ]:
sig, bkg, [bbcut, ttcut] = sig_bkg_opt("2023BPix", gridsize=3, plot=True)

In [ ]:
# plt.rcdefaults()

In [ ]:
rocs["2023BPix"]["tautau"][f"{taukey}vsQCDTop"]["thresholds"]

In [ ]:
bg_combined = 0
sig_combined = 0

results = {}

for year in years:
    sig_yield, bkg_yield, [bbcut, ttcut] = sig_bkg_opt(year, gridsize=30, plot=True)
    results[year] = as_df(sig_yield, bkg_yield, [year])
    bg_combined += bkg_yield
    sig_combined += sig_yield

results["Combined"] = as_df(sig_combined, bg_combined, years)

In [ ]:
results_df = pd.concat(results, axis=0)
results_df.index = results_df.index.droplevel(1)
print(results_df.T.to_markdown())
results_df.T.to_csv(plot_dir / f"{years}-results.csv")

| Muons                   |        2023 |    2023BPix |    Combined |
|:-------------------|------------:|------------:|------------:|
| Sig_Yield          |   0.0305736 |   0.0150834 |   0.0456569 |
| BG_Yield           |   1         |   0         |   1         |
| Limit              |  65.416     |   0         |  43.805     |
| Limit_scaled_22_23 | nan         | nan         | nan         |
| Limit_scaled_22_24 |  11.6528    |   0         |   9.66919   |
| Limit_scaled_Run3  |   8.36266   |   0         |   6.93912   |

| **Period (Electrons)**                        | **2022**  | **2022EE** | **Combined (2022, 2022EE)** |
|-------------------------------------|----------:|----------:|---------------------------:|
| **Sig yield**                       | 0.015     | 0.046     | 0.061                     |
| **BG yield**                        | 2.000     | 7.000     | 9.000                     |
| **limit**                           | 190.635   | 115.250   | 98.765                    |
| **limit scaled to 22-23 channels**  | 39.654    | 43.576    | 42.621                    |
| **limit scaled to 22-24 channels**  | 22.821    | 25.078    | 24.529                    |
| **limit scaled to Run 3 channels**  | 16.378    | 17.997    | 17.603                    |


| **Period (Muons)**                        | **2022**  | **2022EE** | **Combined (2022, 2022EE)** |
|-------------------------------------|---------:|----------:|---------------------------:|
| **Sig yield**                       | 0.015    | 0.047     | 0.062                     |
| **BG yield**                        | 0.000    | 3.000     | 3.000                     |
| **limit**                           | 0.000    | 73.528    | 55.506                    |
| **limit scaled to 22-23 channels**  | 0.000    | 27.801    | 23.953                    |
| **limit scaled to 22-24 channels**  | 0.000    | 16.000    | 13.785                    |
| **limit scaled to Run 3 channels**  | 0.000    | 11.482    | 9.893                     |


| **Period (Full hadronic)**                        | **2022**  | **2022EE** | **Combined (2022, 2022EE)** |
|-------------------------------------|---------:|----------:|---------------------------:|
| **Sig yield**                       | 0.032    | 0.104     | 0.136                     |
| **BG yield**                        | 1.000    | 3.000     | 4.000                     |
| **limit**                           | 62.243   | 33.457    | 29.483                    |
| **limit scaled to 22-23 channels**  | 12.947   | 12.650    | 12.723                    |
| **limit scaled to 22-24 channels**  | 7.451    | 7.280     | 7.322                     |
| **limit scaled to Run 3 channels**  | 5.347    | 5.225     | 5.255                     |
